In [13]:
import pandas as pd
import numpy as np
from scipy import stats

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:
AUCs.txt

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо. 

In [8]:
data = pd.read_csv('AUCs.txt', sep='\t')
data.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


Критерий знаковых рангов Уилкоксона

$H_0\colon med\left(X_1-X_2\right)=0,$

$H_1\colon med\left(X_1-X_2\right)\neq0$

In [33]:
print data.columns[1::]

Index([u'C4.5', u'C4.5+m', u'C4.5+cf', u'C4.5+m+cf'], dtype='object')


In [66]:
#stats.wilcoxon(weight_data.After, weight_data.Before)
cols = data.columns[1:]
p_values = []
for i in range(1, len(cols)):
    for j in range(i):
        (stat, pvalue) = stats.wilcoxon(data[[cols[i]]].values.ravel(),data[[cols[j]]].values.ravel())
        p_values.append([cols[i] + ' - ' + cols[j],pvalue])

print p_values
vals = np.array([x[1] for x in p_values])
min_index = np.argmin(vals)
print 'Answer: ', p_values[min_index]

[['C4.5+m - C4.5', 0.01075713311978963], ['C4.5+cf - C4.5', 0.86126233009534803], ['C4.5+cf - C4.5+m', 0.046332729793395394], ['C4.5+m+cf - C4.5', 0.015906444101703374], ['C4.5+m+cf - C4.5+m', 0.32782567584464062], ['C4.5+m+cf - C4.5+cf', 0.022909099354356588]]
Answer:  ['C4.5+m - C4.5', 0.01075713311978963]


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [67]:
print vals[np.where(vals < 0.05)[0]]
print 'Answer: ', len(vals[np.where(vals < 0.05)[0]])

[ 0.01075713  0.04633273  0.01590644  0.0229091 ]
Answer:  4


Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

## Поправка на множественную проверку

### Метод Холма

In [64]:
from statsmodels.sandbox.stats.multicomp import multipletests 


In [71]:
pval_data = pd.DataFrame.from_records(p_values)
pval_data.columns = ['method', 'p']
pval_data.head()
(pval_data.p < 0.05).value_counts()

True     4
False    2
Name: p, dtype: int64

In [73]:
reject, p_corrected, a1, a2 = multipletests(pval_data.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 
pval_data['p_corrected'] = p_corrected
pval_data['reject'] = reject
pval_data.head()

,method,p,p_corrected,reject
0,C4.5+m - C4.5,0.010757,0.064543,False
1,C4.5+cf - C4.5,0.861262,0.861262,False
2,C4.5+cf - C4.5+m,0.046333,0.138998,False
3,C4.5+m+cf - C4.5,0.015906,0.079532,False
4,C4.5+m+cf - C4.5+m,0.327826,0.655651,False


Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга? 

### Метод Бенджамини-Хохберга

In [75]:
reject, p_corrected, a1, a2 = multipletests(pval_data.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
pval_data['p_corrected'] = p_corrected
pval_data['reject'] = reject
pval_data.head()

,method,p,p_corrected,reject
0,C4.5+m - C4.5,0.010757,0.045818,True
1,C4.5+cf - C4.5,0.861262,0.861262,False
2,C4.5+cf - C4.5+m,0.046333,0.069499,False
3,C4.5+m+cf - C4.5,0.015906,0.045818,True
4,C4.5+m+cf - C4.5+m,0.327826,0.393391,False
